# 論文・論拠データを整理する
投稿する論文とその論拠データを整理するタスクです。<br>
投稿先の規定に従って論文を修正し、論文の内容に即した形で論拠データを整理してください。

## 論文が投稿先の規定に適合することを確認する
投稿先の規定を確認し、以下の手順に従って作業してください。

### 1. リサーチフローのデータをコピーする
親サブフローになっている論文執筆サブフローから論文をこのサブフローにコピーして下さい。<br>
「リサーチフローのデータをコピーする」セルを実行することで親サブフローになっている論文執筆サブフローのフォルダと論文データのコピーを保存するためのフォルダを表示することができます。<br>
既に同じ論文執筆サブフローから査読サブフローを作成していた場合、そこに保存されている論文を使用することができます。<br>
以下の図の説明に従ってデータのコピーを行ってください。<br>
｛ダウンロードからアップロードまでの流れを説明する図｝<br>
親サブフローを変更する場合はメインメニューに戻り親子関係を変更した後、このタスクを実行してください。<br>
この作業は投稿先に合わせた修正が必要ない場合も実行してください。

In [ ]:
# リサーチフローのデータをコピーする
import os

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config
from library.utils.error import NotFoundSubflowDataError
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.widgets import MessageBox


notebook_name = 'organize_paper_and_argument_data.ipynb'


class Display(TaskDirector):
    """実験サブフローと論拠データのフォルダを表示するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス
    """

    def __init__(self, working_path: str) -> None:
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        self.reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()

    @TaskDirector.task_cell("1")
    def generate_folder_buttons(self):
        """データフォルダを表示するためのボタンを生成するメソッドです。"""

        self.doing_task()

        _, subflow_id = get_subflow_type_and_id(self.working_path)
        parents_list = self.get_parent_information(subflow_id)


        # 表示する論文執筆サブフローが存在しない場合はメッセージを表示する
        if self._msg_output.has_message():
            self._form_section.append(self._msg_output)

        else:
            old_part= os.path.join("review", subflow_id)
            # 取得した論文執筆サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in parents_list.items():
                new_part = os.path.join("writing", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path, "paper")
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config("organize_argument_data", "tittle_writing")}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        try:
            review_subflow = self.get_children_information(parents_list, subflow_id)
            # 取得した査読サブフローのフォルダを開くボタンの作成
            for review_name, review_id in review_subflow.items():
                new_working_path = self.working_path.replace(subflow_id, review_id)
                button = open_data_folder(new_working_path, "paper")
                card = pn.Card(
                    button,
                    title = f"{msg_config("organize_argument_data", "tittle_review")}{review_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        except NotFoundSubflowDataError:
            pass


        # 論文を保存するフォルダを開くボタンの作成
        pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
        button = open_data_folder(self.working_path, "paper")
        card = pn.Card(
            button,
            title = f"{msg_config("organize_argument_data", "tittle_paper_folder")}",
            width = 550,
            height = 100,
            css_classes = ["custom-card"]
        )
        self._form_section.append(card)

        self.done_task()

        pn.extension()
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_information(self, current_subflow: str) -> dict:
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            current_subflow (str): 現在作業を行っているサブフローのID

        Returns:
            dict: 親サブフローとなっている実験サブフローの情報

        """

        # 全ての親サブフローIDを取得
        parent_ids = self.reserch_flow_status_operater.get_parent_ids(4, current_subflow)

        # 論文執筆サブフローのみを取得
        parent_subflow = {}
        for parent_id in parent_ids:
            try:
                parent_name = self.reserch_flow_status_operater.get_flow_name(3, parent_id)
                parent_subflow[parent_name] = parent_id
            except NotFoundSubflowDataError:
                continue

        # 親サブフローに論文執筆サブフローが存在しない場合はメッセージを格納
        if not parent_subflow:
            self._msg_output.update_warning(msg_config.get("organize_argument_data", "not_found_writing_subflow"))

        return parent_subflow

    def get_children_information(self, parent_subflow: dict, subflow_id: str) -> dict:
        """"子サブフローに関する情報を取得するメソッドです。"""

        child_subflow = {}
        for parent_id in parent_subflow.values():
            try:
                child_subflow.update(self.reserch_flow_status_operater.get_children_id_and_name(4, parent_id))

            except NotFoundSubflowDataError:
                continue

        if len(child_subflow) > 1:
            working_subflow_name = self.reserch_flow_status_operater.get_flow_name(4, subflow_id)
            del child_subflow[working_subflow_name]
            return child_subflow

        else:
            raise NotFoundSubflowDataError

Display(os.path.abspath('__file__')).generate_folder_buttons()

### 2．論文を修正する
コピーした論文が投稿先の規定に適合していない場合は論文の修正を行ってください。<br>
論文の修正後は必ずPDFとして出力してください。<br>
論文の修正をリサーチフロー外で行う場合は、修正後に「フォルダを表示する」セルを実行して直接アップロードしてください。<br>
<br>
論文を執筆する際にリサーチフローに保存した草稿や論拠データを閲覧したい場合は以下のセルを実行して下さい。

In [ ]:
# リサーチフローのデータをコピーする
import os

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config
from library.utils.error import NotFoundSubflowDataError
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.widgets import MessageBox


notebook_name = 'organize_paper_and_argument_data.ipynb'


class Display(TaskDirector):
    """実験サブフローと論拠データのフォルダを表示するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス
    """

    def __init__(self, working_path: str) -> None:
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        self.reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()

    @TaskDirector.task_cell("1")
    def generate_folder_buttons(self):
        """データフォルダを表示するためのボタンを生成するメソッドです。"""

        self.doing_task()

        _, subflow_id = get_subflow_type_and_id(self.working_path)
        parents_list = self.get_parent_information(subflow_id)


        # 表示する論文執筆サブフローが存在しない場合はメッセージを表示する
        if self._msg_output.has_message():
            self._form_section.append(self._msg_output)

        else:
            old_part= os.path.join("review", subflow_id)
            # 取得した論文執筆サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in parents_list.items():
                new_part = os.path.join("writing", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path)
                button = open_data_folder(new_working_path, "draft")
                card = pn.Card(
                    button,
                    title = f"{msg_config("organize_argument_data", "tittle_writing")}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        # 論拠データフォルダを開くボタンの作成
        pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
        button = open_data_folder(self.working_path, "argument_data")
        card = pn.Card(
            button,
            title = f"{msg_config("organize_argument_data", "tittle_experimen")}",
            width = 550,
            height = 100,
            css_classes = ["custom-card"]
        )
        self._form_section.append(card)

        self.done_task()

        pn.extension()
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_information(self, current_subflow: str) -> dict:
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            current_subflow (str): 現在作業を行っているサブフローのID

        Returns:
            dict: 親サブフローとなっている実験サブフローの情報

        """

        # 全ての親サブフローIDを取得
        parent_ids = self.reserch_flow_status_operater.get_parent_ids(4, current_subflow)

        # 論文執筆サブフローのみを取得
        parent_subflow = {}
        for parent_id in parent_ids:
            try:
                parent_name = self.reserch_flow_status_operater.get_flow_name(3, parent_id)
                parent_subflow[parent_name] = parent_id
            except NotFoundSubflowDataError:
                continue

        # 親サブフローに論文執筆サブフローが存在しない場合はメッセージを格納
        if not parent_subflow:
            self._msg_output.update_warning(msg_config.get("organize_argument_data", "not_found_writing_subflow"))

        return parent_subflow

Display(os.path.abspath('__file__')).generate_folder_buttons()

#### リサーチフロー上での修正方法<br>
1. 「フォルダを表示する」セルを実行することで論文が保存されたフォルダが表示されます。<br>
2. 図の指示に従って論文ファイルを編集してください。<br>
｛論文ファイルの編集方法を実際の画面を使って説明する図｝（アップロードの方法も記載する）<br>
<span style="color:red">※ 論文ファイルの編集後は必ず保存してからファイルを閉じてください。</span>　

In [ ]:
# フォルダを表示する
import os

import panel as pn
from IPython.core.display import Javascript
from IPython.display import display

from library.utils.access import open_data_folder


folder_name = 'paper'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

3. 編集したtex形式の論文をPDFとして出力する<br>
terminalで以下のコマンドを実行することで.tex形式のファイルをPDFで出力することができます。<br>
｛PDF化のコマンド｝<br>
以下のセルを実行することでterminalを使用せずにPDF化することも可能です。
※同名のtexファイルをPDF化する場合は自動的に上書きされるので注意してください。

In [ ]:
# texファイルをPDFとして出力する
import os
from library.utils.setting import get_data_dir

fail_name ="" #　ここにＰＤＦ化したいファイルを入力する(.texまで含む)

dir_name = get_data_dir(os.path.abspath('__file__'))
full_path = os.path.join(dir_name, 'papper', fail_name)

# PDF化するコマンド
!pdelatex {full_path}

### 3．論文のメタデータ修正
論文に登録するPDFメタデータを入力するためのフォームを表示します。<br>
「登録する」ボタンを押下することで論文執筆サブフローで保存した論文のPDFメタデータを入力した値で更新します。

In [ ]:
# メタデータの登録フォームを表示する

import os

from library.utils.pdf_metadata import PdfMetaData
from library.utils.setting.research_flow_status import get_data_dir
from library.task_director import TaskDirector

notebook_name = 'register_paper_metadata.ipynb'

class RegisterPaperMetadata(TaskDirector):

    def __init__(self, working_path:str):

        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def register_metadata(self):

        self.doing_task()

        data_dir = get_data_dir(self.working_path)
        data_dir = os.path.join(data_dir, 'paper')

        pdf_metadata = PdfMetaData
        pdf_metadata.generate_pdf_select_box(data_dir)

        self.done_task()

RegisterPaperMetadata(os.path.abspath('__file__')).register_metadata()

## 論拠データがジャーナルの規定に適合することを確認する
投稿先の規定を確認し、査読者に公開する論拠データを以下の手順に従って整理して下さい。<br>
査読者に論拠データを公開しない場合はこの作業はスキップして下さい。

### 1．リサーチフローデータをコピーする
親サブフローになっている論文執筆サブフローから必要な論拠データをこのサブフローにコピーして下さい。<br>
リサーチフローのデータをコピーする」セルを実行することで親サブフローになっている論文執筆サブフローの論拠データフォルダとコピーしたデータを保存するためのフォルダを表示することができます。<br>
既に同じ論文執筆サブフローから査読サブフローを作成していた場合、そこに保存されている論拠データを使用することができます。<br>
この作業は投稿先に合わせた修正が必要ない場合も実行してください。

In [ ]:
# リサーチフローのデータをコピーする
import os

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config
from library.utils.error import NotFoundSubflowDataError
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.widgets import MessageBox


notebook_name = 'organize_paper_and_argument_data.ipynb'


class Display(TaskDirector):
    """実験サブフローと論拠データのフォルダを表示するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス
    """

    def __init__(self, working_path: str) -> None:
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        self.reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()

    @TaskDirector.task_cell("1")
    def generate_folder_buttons(self):
        """データフォルダを表示するためのボタンを生成するメソッドです。"""

        self.doing_task()

        _, subflow_id = get_subflow_type_and_id(self.working_path)
        parents_list = self.get_parent_information(subflow_id)


        # 表示する論文執筆サブフローが存在しない場合はメッセージを表示する
        if self._msg_output.has_message():
            self._form_section.append(self._msg_output)

        else:
            old_part= os.path.join("review", subflow_id)
            # 取得した論文執筆サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in parents_list.items():
                new_part = os.path.join("writing", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path)
                button = open_data_folder(new_working_path, "argument_data")
                card = pn.Card(
                    button,
                    title = f"{msg_config("organize_argument_data", "tittle_writing")}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        try:
            review_subflow = self.get_children_information(parents_list, subflow_id)
            # 取得した査読サブフローのフォルダを開くボタンの作成
            for review_name, review_id in review_subflow.items():
                new_working_path = self.working_path.replace(subflow_id, review_id)
                button = open_data_folder(new_working_path, "argument_data")
                card = pn.Card(
                    button,
                    title = f"{msg_config("organize_argument_data", "tittle_review")}{review_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        except NotFoundSubflowDataError:
            pass


        # 論拠データフォルダを開くボタンの作成
        pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
        button = open_data_folder(self.working_path, "argument_data")
        card = pn.Card(
            button,
            title = f"{msg_config("organize_argument_data", "tittle_argument_data")}",
            width = 550,
            height = 100,
            css_classes = ["custom-card"]
        )
        self._form_section.append(card)

        self.done_task()

        pn.extension()
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_information(self, current_subflow: str) -> dict:
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            current_subflow (str): 現在作業を行っているサブフローのID

        Returns:
            dict: 親サブフローとなっている実験サブフローの情報

        """

        # 全ての親サブフローIDを取得
        parent_ids = self.reserch_flow_status_operater.get_parent_ids(4, current_subflow)

        # 論文執筆サブフローのみを取得
        parent_subflow = {}
        for parent_id in parent_ids:
            try:
                parent_name = self.reserch_flow_status_operater.get_flow_name(3, parent_id)
                parent_subflow[parent_name] = parent_id
            except NotFoundSubflowDataError:
                continue

        # 親サブフローに論文執筆サブフローが存在しない場合はメッセージを格納
        if not parent_subflow:
            self._msg_output.update_warning(msg_config.get("organize_argument_data", "not_found_writing_subflow"))

        return parent_subflow

    def get_children_information(self, parent_subflow: dict, subflow_id: str) -> dict:
        """"子サブフローに関する情報を取得するメソッドです。"""

        child_subflow = {}
        for parent_id in parent_subflow.values():
            try:
                child_subflow.update(self.reserch_flow_status_operater.get_children_id_and_name(4, parent_id))

            except NotFoundSubflowDataError:
                continue

        if len(child_subflow) > 1:
            working_subflow_name = self.reserch_flow_status_operater.get_flow_name(4, subflow_id)
            del child_subflow[working_subflow_name]
            return child_subflow

        else:
            raise NotFoundSubflowDataError

Display(os.path.abspath('__file__')).generate_folder_buttons()

### 2．論拠データを整理する
コピーした論拠データが投稿先の規定に適合していない場合は以下のセルを実行し、「表示する」ボタンから論拠データのフォルダを開き、整理を行ってください。

In [ ]:
# フォルダを表示する
import os

import panel as pn
from IPython.core.display import Javascript
from IPython.display import display

from library.utils.access import open_data_folder


folder_name = 'argument_data'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 論拠データにメタデータを付与する
論拠データのメタデータ登録を行ってください。<br>
セルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
Gakunin RDMにメタデータを登録する場合はGakunin RDM上から手動で行ってください。

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_register_metadata():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/metadata.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_register_metadata_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_register_metadata()

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※再現性を確認する機能は現在開発中です。

### メタデータを検証する
論拠データのメタデータの検証を行ってください<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。

In [ ]:
# 検証するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_validate():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/validate.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_validate_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## GakuNin RDMに保存する

In [ ]:
# GakuNin RDMに保存する
import os
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import path_config
from library.task_director import TaskDirector
from library.utils.setting import get_data_dir

script_file_name = 'save_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        data_dir = get_data_dir(self.working_path)
        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        source = [os.path.join(data_dir, 'argument_data'), os.path.join(data_dir, 'paper'), research_flow_status_file_path]
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()